In [ ]:
import os
import csv
import pandas as pd
import pandas.io.sql as psql
import datetime as dt
import numpy as np
import warnings


import gspread
import gspread_dataframe as gd
import psycopg2 as pg

warnings.filterwarnings('ignore')

DbInternal = pg.connect(DbInternal)
DbMain = pg.connect(DbMain)

In [ ]:
# all countries
# sel_countries = ["Australia", "Singapore",  "Canada", "Malaysia", "Taiwan",  "Indonesia", "New Zealand", "Japan", "Hong Kong", "South Korea", "Philippines", "United Kingdom", "United States", "Vietnam", "Thailand"]
sel_countries = ["Taiwan"]
def combine_str(items):
    result = "', '".join(str(item) for item in items[:-1]) 
    return "'" + result + "', '" + str(items[-1]) + "'" 

sel_countries = "(" + combine_str(tuple(sel_countries)) + ")"

In [ ]:
today_date = dt.date.today()
today_date = dt.datetime.strftime(today_date, '%Y-%m-%d')

In [ ]:
top_views = psql.read_sql('''WITH ranked_products AS (
    SELECT
        country,
        product_id,
        SUM(product_views_total) AS total_views,
        SUM(product_clicks_total) AS total_clicks,
        SUM(product_views_unique) AS total_views_unique,
        SUM(product_clicks_unique) AS total_clicks_unique,
        ROW_NUMBER() OVER(PARTITION BY country ORDER BY SUM(product_views_total) DESC) AS views_rank
    FROM
        db_product_views_ct
    WHERE
        month >= CURRENT_DATE - INTERVAL '90 days' AND
        country in %s
    GROUP BY
        country,
        product_id
),
product_names AS (
    SELECT DISTINCT ON (product_id)
        product_id,
        name
    FROM
        db_product_views_ct
)
SELECT
    rp.country,
    pn.name AS product_name,
    pn.product_id,
    rp.total_views,
    rp.total_clicks,
    rp.total_views_unique,
    rp.total_clicks_unique
FROM
    ranked_products rp
LEFT JOIN
    product_names pn ON rp.product_id = pn.product_id
WHERE
    rp.views_rank <= 10;''' % (sel_countries), DbInternal)

In [ ]:
product_ids = tuple(top_views['product_id'].astype(str).unique())
product_ids = "(" + combine_str(product_ids) + ")"

In [ ]:
transactions = psql.read_sql('''
    SELECT co.name as country, b.product_id, 
    COUNT(b.id) as transactions,
    sum(COALESCE(list_price_local/seller_currency_rate,0)
    + COALESCE(fee_delivery/buyer_currency_rate,0)
    + COALESCE(fee_delivery_discount/buyer_currency_rate,0)
    + COALESCE(fee_delivery_insurance/buyer_currency_rate,0)
    + COALESCE(fee_delivery_instant/buyer_currency_rate,0)
    + case when COALESCE(offer_price_local/buyer_currency_rate,0) > COALESCE(list_price_local/seller_currency_rate,0)
        then COALESCE(offer_price_local/buyer_currency_rate,0) - COALESCE(list_price_local/seller_currency_rate,0)
        else 0 end
    + COALESCE(fee_add_on/buyer_currency_rate,0)
    + COALESCE(fee_processing_buy/buyer_currency_rate,0) 
    - COALESCE(total_tax_reportable/buyer_currency_rate,0)) as gmv
    FROM transaction_buyer b
    left join transaction_seller s
    on b.transaction_seller_id = s.id
    left join country co on b.buyer_country_id = co.id
    left join product p on b.product_id = p.id
    where date(b.created_at + interval '8 hours' ) >= CURRENT_DATE - INTERVAL '90 days'
    and b."type" = 'transaction'
    and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
    and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
    'to_procure','bidding','to_complete_buy', 'contact_platform')
    or COALESCE(b.nature,'default') not in  ('Pre Order'))
    and b.payment_method!='credit'
    and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
    and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
    'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
    'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
    'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
    and (seller_id not in (12409, 12698) or seller_id is null)
    and b.product_id in %s
    and co.name in %s
    GROUP BY 1,2
    ORDER BY 1,2
    ''' % (product_ids, sel_countries), DbMain)
transactions

In [ ]:
top_views_purchase = pd.merge(top_views, transactions, on = ['country', 'product_id'], how = 'left')
top_views_purchase = top_views_purchase.fillna(0)

In [ ]:
def sort_within_group(group):
    return group.sort_values(by='total_views', ascending=False)

top_views_purchase_sorted = top_views_purchase.groupby('country').apply(sort_within_group)
top_views_purchase_sorted = top_views_purchase_sorted.reset_index(drop=True)

In [ ]:
top_views_purchase.groupby('product_name')['total_views'].sum().reset_index().sort_values('total_views', ascending = False).head(20)
top_views_purchase.groupby('product_name')['transactions'].sum().reset_index().sort_values('transactions', ascending = False).head(15)

In [ ]:
first_buyer = psql.read_sql('''
    with first_buy as (select
    buyer_id ,min(b.created_at + interval '8 hours') as first_order_date,
    min(b.id) as first_order_id
    from transaction_buyer b
    left join transaction_seller s
    on b.transaction_seller_id = s.id
    left join country co 
    on b.buyer_country_id = co.id
    where
    date(b.created_at + interval '8 hours' ) >= CURRENT_DATE - INTERVAL '90 days' and
    b."type" = 'transaction' and
    b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed') and co.name in %s 
    and b.payment_method!='credit'
    and buyer_id not in (12409, 20095, 1804, 19022, 87853) and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
    'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
    'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
    'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
    and (seller_id not in (12409, 12698) or seller_id is null)
    and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
    'to_procure','bidding','to_complete_buy', 'contact_platform')
    or COALESCE(b.nature,'default')  not in  ('Pre Order'))
    group by 1
    order by 1)

    select p.name,
    count(distinct f.buyer_id) as new_buyers
    from first_buy f
    left join transaction_buyer b
    on b.id = f.first_order_id
    left join transaction_seller s
    on b.transaction_seller_id = s.id
    left join product p 
    ON b.product_id = p.id
    group by 1
    order by 1 
    ''' % (sel_countries), DbMain)

In [ ]:
first_buyer.sort_values('new_buyers', ascending = False).head(20)

In [ ]:
top_views_purchase_sorted.groupby('product_name')['transactions'].sum().reset_index().sort_values('transactions', ascending = False).head(15)

In [ ]:
country_views = top_views_purchase.groupby('country')['total_views'].sum()

sorted_countries = country_views.sort_values(ascending=False).index

top_views_purchase_sorted = top_views_purchase.set_index('country').loc[sorted_countries].reset_index()

In [ ]:
gc1 = gspread.service_account('service_account.json')

sh = gc1.open_by_url('https://docs.google.com/spreadsheets/d/1VrQBCT-DC06QGvrmNo9rpjHnvnxHVHVlaQ2rr2HIXTs/edit#gid=0')
sh1 = sh.worksheet('Data')

gd.set_with_dataframe(sh1, top_views_purchase_sorted)

In [ ]:
DbMain.close()